In [13]:
import os
import PyPDF2
import pdfplumber
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import pandas as pd
import joblib

In [14]:
def extract_text_and_features_from_pdf(file_path):
    text = ""
    num_pages = 0
    has_table = False
    keyword_count = {"factura": 0, "total": 0, "contrato": 0, "cláusula": 0, "firma": 0, "cufe": 0}
    with pdfplumber.open(file_path) as pdf:
        num_pages = len(pdf.pages)
        for page in pdf.pages:
            text += page.extract_text()
            if page.extract_tables():
                has_table = True
    for keyword in keyword_count.keys():
        keyword_count[keyword] = text.lower().count(keyword)
    features = [num_pages, int(has_table)] + list(keyword_count.values())
    return features

In [17]:
labels = []
features_list = []
for category in ['factura', 'contrato']:
    folder_path = os.path.join('train', category)
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        features = extract_text_and_features_from_pdf(file_path)
        labels.append(category)
        features_list.append(features)

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

In [18]:
df_feature = pd.DataFrame(features_list, columns=['num_pages', 'has_table', 'factura', 'total', 'contrato', 'cláusula', 'firma','cufe'])
df_label = pd.DataFrame(labels, columns=['category'])
df = pd.concat([df_feature, df_label], axis=1)
df

,num_pages,has_table,factura,total,contrato,cláusula,firma,cufe,category
0,2,1,5,4,0,0,2,1,factura
1,1,1,5,3,0,0,2,1,factura
2,1,1,5,3,0,0,2,1,factura
3,2,1,5,2,0,0,2,1,factura
4,2,1,6,5,0,0,0,0,factura
5,2,1,6,5,0,0,0,0,factura
6,1,1,7,2,0,0,0,1,factura
7,1,1,7,2,0,0,0,1,factura
8,1,1,7,2,0,0,0,1,factura
9,1,1,7,2,0,0,0,1,factura


In [20]:
X = df.drop('category', axis=1)
y = df['category']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
classifier = RandomForestClassifier()
classifier.fit(X_train, y_train)
joblib.dump(classifier, 'modelo_entrenado.pkl')

['modelo_entrenado.pkl']

In [22]:
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Precisión del modelo: {accuracy:.2f}")

Precisión del modelo: 1.00


In [23]:
report = classification_report(y_test, y_pred)
print("\nReporte de clasificación:")
print(report)


Reporte de clasificación:
              precision    recall  f1-score   support

    contrato       1.00      1.00      1.00         3
     factura       1.00      1.00      1.00         6

    accuracy                           1.00         9
   macro avg       1.00      1.00      1.00         9
weighted avg       1.00      1.00      1.00         9

